In [233]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "browser"

In [205]:
leeg22 = pd.read_csv('../data/league_2022.csv')
leeg23 = pd.read_csv('../data/league_2023.csv')
leeg24 = pd.read_csv('../data/league_2024.csv')

league = pd.concat([leeg22, leeg23, leeg24])
league.head()

/Users/joycehu/miniforge3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning:

Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.



,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0


# Data Cleaning

In [206]:
# make copy 
pre_cleaning = league.copy()

# binary variable for missing patch
league['patch_missing'] = (league['patch'].isna()).astype(int)

# impute missing patches
patches = {
    '2022-08-11 08:37:33':12.15, 
    '2022-08-12 06:08:39':12.15, 
    '2022-08-12 07:04:40':12.15, 
    '2022-08-12 07:52:33':12.15, 
    '2022-08-12 08:51:49':12.15, 
    '2022-08-12 09:58:13':12.15, 
    '2022-08-13 06:08:31':12.15, 
    '2022-08-13 06:55:35':12.15, 
    '2022-08-13 06:08:31':12.15,
    '2022-08-13 06:55:35':12.15,
    '2022-08-13 07:47:42':12.15,
    '2023-04-05 09:17:28':13.07,
    '2023-04-05 10:18:03':13.07, 
    '2023-04-05 11:23:10':13.07,
    '2023-04-05 12:21:13':13.07,
    '2023-04-05 13:10:18':13.07,
    '2023-05-08 06:12:19':13.09,
    '2023-05-08 07:07:13':13.09,
    '2023-05-08 08:12:21':13.09,
    '2023-05-08 09:06:19':13.09,
    '2023-05-08 10:08:36':13.09,
    '2024-02-21 06:02:57':14.03,
    '2024-02-21 06:56:52':14.03, 
    '2024-02-21 07:43:27':14.03,
    '2024-02-21 08:31:33':14.03, 
    '2024-02-21 09:17:02':14.03
}
league['patch'] = league['patch'].fillna(league['date'])
league['patch'] = league['patch'].replace(patches)
print(league['patch'].unique())

# filter out summary rows 
league = league[(league['participantid'] != 100) & (league['participantid'] != 200)]

# create a binary variable for before and after Patch 13.9
league['post_rework'] = (league['patch'] >= 13.09).astype(int)

# create a binary variable for whether neeko played in the game
league['neeko'] = (league['champion'] == 'Neeko').astype(int)


[12.01 12.02 12.03 12.04 12.05 12.06 12.07 12.08 12.09 12.1  12.11 12.12
 12.13 12.14 12.15 12.16 12.17 12.18 12.19 12.2  12.21 12.23 13.01 13.03
 13.04 13.05 13.06 13.07 13.08 13.09 13.1  13.11 13.12 13.13 13.14 13.15
 13.16 13.17 13.18 13.19 13.2  13.21 13.22 13.24 14.01 14.02 14.03 14.04
 14.05]


# Missingness

In [109]:
league[league['patch'].notna()]['year'].value_counts(normalize=True)

2022    0.467927
2023    0.406535
2024    0.125538
Name: year, dtype: float64

In [110]:
league[league['patch'].isna()]['year'].value_counts(normalize=True)

2023    0.416667
2022    0.375000
2024    0.208333
Name: year, dtype: float64

In [ ]:
gender_dist = (
    heights_mcar
    .assign(child_missing=heights_mcar['child'].isna())
    .pivot_table(index='gender', columns='child_missing', aggfunc='size')
)

# Added just to make the resulting pivot table easier to read.
gender_dist.columns = ['child_missing = False', 'child_missing = True']

gender_dist = gender_dist / gender_dist.sum()
gender_dist

In [213]:
piv = league.pivot_table(index='league', columns='patch_missing', aggfunc='size').fillna(0)
piv

piv = piv / piv.sum()
piv

patch_missing,0,1
league,,
AL,0.008576,0.000000
CBLOL,0.022236,0.000000
CBLOLA,0.020994,0.000000
CDF,0.005472,0.000000
CT,0.002678,0.000000
DCup,0.005278,0.000000
DDH,0.011448,0.000000
EBL,0.015911,0.000000
EL,0.006830,0.000000


In [220]:
league['league'].nunique() / league['patch_missing'].sum()

0.23333333333333334

In [216]:
league.groupby('league')['patch_missing'].sum() / league.groupby('league')['patch_missing'].sum().sum()

league
AL            0.000000
CBLOL         0.000000
CBLOLA        0.000000
CDF           0.000000
CT            0.000000
DCup          0.000000
DDH           0.000000
EBL           0.000000
EL            0.000000
EM            0.000000
EPL           0.000000
ESLOL         0.000000
EUM           0.000000
GL            0.000000
GLL           0.000000
HC            0.000000
HM            0.000000
IC            0.000000
LAS           0.000000
LCK           0.000000
LCKC          0.000000
LCL           0.000000
LCO           0.000000
LCS           0.000000
LCSA          0.000000
LDL           0.791667
LEC           0.000000
LFL           0.000000
LFL2          0.000000
LHE           0.000000
LIT           0.000000
LJL           0.000000
LJLA          0.000000
LLA           0.000000
LMF           0.000000
LPL           0.208333
LPLOL         0.000000
LRN           0.000000
LRS           0.000000
LVP SL        0.000000
MSI           0.000000
NACL          0.000000
NEXO          0.000000
NLC 

In [207]:
league['patch_missing'].sum()

240

### Permutation Test:

Is there a higher proportion of games with a Neeko in it after the Patch 13.9 rework?

Null: There is no difference in proportion of games with Neeko before and after the Patch 13.9 rework.


Alternative: There is a higher proportion of games with Neeko after the Patch 13.9 rework. 


Test Statistic: Difference in Proportion


Significance Level: 0.05


Result p-value:


In [190]:
# create a new df where each observation is a game (game id becomes unique)
rework = league.groupby('gameid').agg({'post_rework': 'first', 'neeko':'sum'})
rework

,post_rework,neeko
gameid,,
10000-10000_game_1,0,0
10000-10000_game_2,0,0
10000-10000_game_3,0,0
10000-10000_game_4,0,0
10000-10000_game_5,0,0
...,...,...
NA1_4493482900,0,0
NA1_4493540863,0,0
NA1_4493591166,0,0


In [230]:
# observed value is the proportion of games with neeko after - before the patch
obs = rework.groupby('post_rework').mean().diff().iloc[-1][0]
obs

0.1404526229251691

In [238]:
n_reps = 500
diffs = []

for _ in range(n_reps):
    
    # shuffle rework column 
    rework['shuffled_rework'] = np.random.permutation(rework['post_rework'])
    
    # compute test statistic 
    difference = rework.groupby('shuffled_rework').mean().diff().iloc[-1][0]
    
    diffs.append(difference)

diffs[:10]

[-0.01002848491989039,
 -0.010748776226661594,
 0.0023965401219124827,
 0.0013161031617557328,
 -0.001565062065329026,
 0.0034769770820692325,
 0.0018563216418341355,
 0.006358142309153991,
 -0.0035458631589497247,
 0.005817923829075589]

In [243]:
# calculate p-value
(np.array(diffs) >= obs).mean()

0.0

### code that isn't important for this: 

In [21]:
by_patch = players['patch'].value_counts().to_frame().reset_index().sort_values('index')
by_patch.head()

,index,patch
13,12.01,8020
7,12.02,9680
11,12.03,8770
2,12.04,13240
3,12.05,12450


In [63]:
by_patch[(by_patch['index'] < 13.09) & (by_patch['index'] >= 12.10)]['patch'].sum() / 10

11669.0

In [64]:
by_patch[by_patch['index'] > 13.09]['patch'].sum() / 10

7420.0

In [65]:
11669 - 7420.0

4249.0

In [6]:
for col in league.columns:
    print(col)

gameid
datacompleteness
url
league
year
split
playoffs
date
game
patch
participantid
side
position
playername
playerid
teamname
teamid
champion
ban1
ban2
ban3
ban4
ban5
pick1
pick2
pick3
pick4
pick5
gamelength
result
kills
deaths
assists
teamkills
teamdeaths
doublekills
triplekills
quadrakills
pentakills
firstblood
firstbloodkill
firstbloodassist
firstbloodvictim
team kpm
ckpm
firstdragon
dragons
opp_dragons
elementaldrakes
opp_elementaldrakes
infernals
mountains
clouds
oceans
chemtechs
hextechs
dragons (type unknown)
elders
opp_elders
firstherald
heralds
opp_heralds
void_grubs
opp_void_grubs
firstbaron
barons
opp_barons
firsttower
towers
opp_towers
firstmidtower
firsttothreetowers
turretplates
opp_turretplates
inhibitors
opp_inhibitors
damagetochampions
dpm
damageshare
damagetakenperminute
damagemitigatedperminute
wardsplaced
wpm
wardskilled
wcpm
controlwardsbought
visionscore
vspm
totalgold
earnedgold
earned gpm
earnedgoldshare
goldspent
gspd
gpr
total cs
minionkills
monsterkills
mon

In [7]:
league[league['gameid'] == 'ESPORTSTMNT01_2690210']['participantid']

0       1
1       2
2       3
3       4
4       5
5       6
6       7
7       8
8       9
9      10
10    100
11    200
Name: participantid, dtype: int64

# Prediction 

Predict whether Neeko's team wins the game based on her stats at 15 minutes and her position (ohe).

In [69]:
#league[pd.isna(league['patch'])]

In [75]:
league[league['game'] == 5]

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
42432,ESPORTSTMNT04_2220386,complete,NaN,ESLOL,2022,Spring,1,2022-03-11 20:12:41,5,12.04,...,101.0,1251.0,1133.0,31.0,1.0,1.0,0.0,0.0,0.0,1.0
42433,ESPORTSTMNT04_2220386,complete,NaN,ESLOL,2022,Spring,1,2022-03-11 20:12:41,5,12.04,...,110.0,251.0,538.0,0.0,1.0,3.0,0.0,1.0,0.0,1.0
42434,ESPORTSTMNT04_2220386,complete,NaN,ESLOL,2022,Spring,1,2022-03-11 20:12:41,5,12.04,...,154.0,-268.0,139.0,-12.0,1.0,0.0,0.0,0.0,0.0,1.0
42435,ESPORTSTMNT04_2220386,complete,NaN,ESLOL,2022,Spring,1,2022-03-11 20:12:41,5,12.04,...,146.0,-1778.0,-1423.0,-35.0,0.0,1.0,1.0,0.0,1.0,0.0
42436,ESPORTSTMNT04_2220386,complete,NaN,ESLOL,2022,Spring,1,2022-03-11 20:12:41,5,12.04,...,23.0,184.0,134.0,8.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23335,LOLTMNT02_61596,complete,NaN,LCO,2024,Split 1,1,2024-03-01 10:38:01,5,14.03,...,151.0,-945.0,-570.0,-5.0,0.0,0.0,2.0,1.0,2.0,0.0
23336,LOLTMNT02_61596,complete,NaN,LCO,2024,Split 1,1,2024-03-01 10:38:01,5,14.03,...,143.0,249.0,137.0,13.0,0.0,0.0,0.0,1.0,0.0,0.0
23337,LOLTMNT02_61596,complete,NaN,LCO,2024,Split 1,1,2024-03-01 10:38:01,5,14.03,...,31.0,-63.0,-265.0,-15.0,0.0,0.0,0.0,0.0,1.0,0.0
23338,LOLTMNT02_61596,complete,NaN,LCO,2024,Split 1,1,2024-03-01 10:38:01,5,14.03,...,581.0,-583.0,-1594.0,-48.0,3.0,6.0,1.0,1.0,1.0,3.0


In [74]:
league['game'].value_counts()

1    195804
2     62292
3     30348
4      9792
5      4272
Name: game, dtype: int64

In [40]:
league[(league['champion'] == 'Neeko') & (league['patch'] >= 13.9)]

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
223,LOLTMNT06_13630,complete,NaN,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,...,136.0,-279.0,330.0,16.0,0.0,1.0,0.0,1.0,0.0,0.0
259,LOLTMNT06_12743,complete,NaN,LEC,2024,Winter,0,2024-01-13 19:24:48,1,14.01,...,147.0,-302.0,-952.0,-16.0,1.0,2.0,1.0,0.0,1.0,0.0
278,LOLTMNT06_12814,complete,NaN,LEC,2024,Winter,0,2024-01-14 16:38:26,1,14.01,...,138.0,-120.0,-56.0,-20.0,1.0,2.0,0.0,0.0,2.0,0.0
307,LOLTMNT06_12845,complete,NaN,LEC,2024,Winter,0,2024-01-14 18:59:08,1,14.01,...,148.0,245.0,4.0,-13.0,2.0,1.0,1.0,0.0,0.0,0.0
391,LOLTMNT02_19357,complete,NaN,LCKC,2024,Spring,0,2024-01-15 08:11:33,1,14.01,...,156.0,-87.0,196.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26978,LOLTMNT06_25430,complete,NaN,EBL,2024,Spring,0,2024-03-07 18:52:01,1,14.04,...,131.0,1086.0,-38.0,-9.0,2.0,0.0,1.0,0.0,1.0,2.0
27098,LOLTMNT06_25442,complete,NaN,EBL,2024,Spring,0,2024-03-07 19:46:23,1,14.04,...,128.0,785.0,-76.0,-8.0,4.0,2.0,2.0,1.0,1.0,1.0
27247,LOLTMNT03_49992,complete,NaN,LVP SL,2024,Spring,0,2024-03-07 20:56:14,1,14.04,...,138.0,168.0,-439.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0
27278,LOLTMNT03_50077,complete,NaN,LPLOL,2024,Spring,0,2024-03-07 21:33:02,1,14.04,...,138.0,-283.0,-795.0,-17.0,1.0,0.0,2.0,1.0,3.0,0.0


In [43]:
neeko = league[league['champion'] == 'Neeko']
neeko.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
16027,ESPORTSTMNT02_2557639,complete,NaN,EL,2022,NaN,0,2022-02-09 00:31:28,1,12.02,...,126.0,381.0,97.0,3.0,1.0,3.0,0.0,2.0,0.0,1.0
21038,NA1_4217310447,complete,NaN,PGC,2022,NaN,0,2022-02-14 23:08:07,1,12.03,...,136.0,-919.0,336.0,-21.0,4.0,1.0,2.0,2.0,0.0,0.0
21192,NA1_4214959149,complete,NaN,PGC,2022,NaN,0,2022-02-14 23:24:23,1,12.03,...,98.0,2273.0,685.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0
28598,ESPORTSTMNT01_2740451,complete,NaN,PGC,2022,NaN,0,2022-02-22 23:41:17,2,12.04,...,84.0,2069.0,713.0,28.0,4.0,9.0,1.0,2.0,2.0,5.0
33679,ESPORTSTMNT01_2743216,complete,NaN,EL,2022,NaN,0,2022-02-28 23:41:41,1,12.04,...,119.0,169.0,396.0,15.0,0.0,0.0,0.0,0.0,2.0,0.0


In [72]:
stats_15 = [col for col in league.columns if col.endswith('15')]
stats_15

league[stats_15]

,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,5025.0,7560.0,135.0,4634.0,7215.0,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,5366.0,5320.0,89.0,4825.0,5595.0,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,5118.0,6942.0,120.0,5593.0,6789.0,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,5461.0,4591.0,115.0,6254.0,5934.0,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,3836.0,3588.0,28.0,3393.0,4085.0,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
league[league['ban1'] == 'Neeko']['patch'].value_counts()

13.11    1842
13.12    1230
13.10     702
13.14     426
13.19     414
13.13     366
14.01     318
14.02     276
14.04      90
13.21      72
13.15      72
13.17      54
13.20      24
13.24      24
12.05      18
13.04      12
12.04      12
13.22       6
12.19       6
12.10       6
14.03       6
Name: patch, dtype: int64

In [42]:
league[league['gameid']=='ESPORTSTMNT01_2690210'].iloc[9:11]

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
9,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,28.0,-443.0,497.0,-7.0,0.0,6.0,2.0,1.0,2.0,2.0
10,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,510.0,107.0,-1617.0,-23.0,5.0,10.0,6.0,6.0,18.0,5.0
